# Перевод всего датасета с английского на русский с помощью модели - Helsinki-NLP/opus-mt-en-ru

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install langdetect

In [4]:
# импорт необходимых библиотек

import pandas as pd
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
df = pd.read_csv('/content/drive/MyDrive/full_df_EN (1).csv')
df

,Название файла,Имя Художника,Описание картины,Название картины
0,Albrecht_Dürer1.jpg,Albrecht Dürer,a drawing of a man with curly hair and a beard,Head of the Mark
1,Albrecht_Dürer2.jpg,Albrecht Dürer,a drawing of a woman holding a baby in her lap,Madonna and child on the grassy bank
2,Albrecht_Dürer3.jpg,Albrecht Dürer,a drawing of a man and woman in a room with a ...,NaN
3,Albrecht_Dürer4.jpg,Albrecht Dürer,a drawing of a man standing next to a woman in...,Peter and John Healing the Cripple at the Gate...
4,Albrecht_Dürer5.jpg,Albrecht Dürer,a drawing of a group of people in a room with ...,Life of the Virgin: The Death of the Virgin
...,...,...,...,...
11423,the-study-of-adam-1.jpg,Michelangelo,a drawing of a man sitting on a rock with his ...,The Study of Adam
11424,the-torment-of-saint-anthony.jpg,Michelangelo,painting of a man riding a dragon with a bird ...,The Torment of Saint Anthony
11425,the-virgin-and-child-with-saint-john-and-angel...,Michelangelo,arafed painting of a woman reading to children...,The Virgin and Child with Saint John and Angel...
11426,various-studies.jpg,Michelangelo,a drawing of a man holding a baby and a woman,Various studies


In [6]:
# Функция для определения языка текста с помощью библиотеки langdetect

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return None

In [7]:
df['language'] = df['Название картины'].apply(detect_language)
df

,Название файла,Имя Художника,Описание картины,Название картины,language
0,Albrecht_Dürer1.jpg,Albrecht Dürer,a drawing of a man with curly hair and a beard,Head of the Mark,en
1,Albrecht_Dürer2.jpg,Albrecht Dürer,a drawing of a woman holding a baby in her lap,Madonna and child on the grassy bank,en
2,Albrecht_Dürer3.jpg,Albrecht Dürer,a drawing of a man and woman in a room with a ...,NaN,None
3,Albrecht_Dürer4.jpg,Albrecht Dürer,a drawing of a man standing next to a woman in...,Peter and John Healing the Cripple at the Gate...,en
4,Albrecht_Dürer5.jpg,Albrecht Dürer,a drawing of a group of people in a room with ...,Life of the Virgin: The Death of the Virgin,en
...,...,...,...,...,...
11423,the-study-of-adam-1.jpg,Michelangelo,a drawing of a man sitting on a rock with his ...,The Study of Adam,en
11424,the-torment-of-saint-anthony.jpg,Michelangelo,painting of a man riding a dragon with a bird ...,The Torment of Saint Anthony,en
11425,the-virgin-and-child-with-saint-john-and-angel...,Michelangelo,arafed painting of a woman reading to children...,The Virgin and Child with Saint John and Angel...,en
11426,various-studies.jpg,Michelangelo,a drawing of a man holding a baby and a woman,Various studies,af


In [8]:
df['language'] = df['language'].replace({'de': 'en', 'fr': 'en', 'af': 'en', 'ro': 'en', 'it': 'en', 'nl': 'en', 'tl': 'en', 'id': 'en', 'es': 'en',
                                         'ca': 'en', 'et': 'en', 'no': 'en', 'da': 'en', 'cy': 'en', 'sw': 'en', 'pt': 'en', 'sv': 'en', 'pl': 'en',
                                         'tr': 'en', 'lt': 'en', 'fi': 'en', 'so': 'en', 'sl': 'en', 'sk': 'en', 'hr': 'en', 'sq': 'en', 'cs': 'en', 'vi': 'en', 'hu': 'en'})

In [10]:
# Создание модели для перевода с английского на русский

tokenizer_en_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model_en_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# Создание модели для перевода с русского на английский

# tokenizer_en_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
# model_en_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

In [11]:
# Перевод текста с помощью моделей
translations = []
for index, row in df.iterrows():
    text = row['Название картины']
    language = row['language']

    if language == 'en':
        encoded_input = tokenizer_en_ru(text, return_tensors='pt', padding=True)
        output = model_en_ru.generate(**encoded_input, num_beams=4, early_stopping=True)
        translated_text = tokenizer_en_ru.decode(output[0], skip_special_tokens=True)
        translations.append(translated_text)
       
    # ЕСЛИ НУЖНО ПЕРЕВЕСТИ С РУССКОГО НА АНГЛИЙСКИЙ, ТО ИСПОЛЬЗУЕТСЯ ЭТО ЦИКЛ
    # if language == 'ru':
    #     encoded_input = tokenizer_en_ru(text, return_tensors='pt', padding=True)
    #     output = model_en_ru.generate(**encoded_input, num_beams=4, early_stopping=True)
    #     translated_text = tokenizer_en_ru.decode(output[0], skip_special_tokens=True)
    #     translations.append(translated_text)

    else:
        translations.append(None)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
# Добавление столбца с переводами в новый датафрейм

df['Название картины RU'] = translations

In [15]:
df.head(15)

,Название файла,Имя Художника,Описание картины,Название картины,language,Название картины RU
0,Albrecht_Dürer1.jpg,Albrecht Dürer,a drawing of a man with curly hair and a beard,Head of the Mark,en,Глава знака
1,Albrecht_Dürer2.jpg,Albrecht Dürer,a drawing of a woman holding a baby in her lap,Madonna and child on the grassy bank,en,Мадонна и ребёнок на банке травы
2,Albrecht_Dürer3.jpg,Albrecht Dürer,a drawing of a man and woman in a room with a ...,NaN,None,None
3,Albrecht_Dürer4.jpg,Albrecht Dürer,a drawing of a man standing next to a woman in...,Peter and John Healing the Cripple at the Gate...,en,Петр и Иоанн Исцеляли калеку у ворот храма
4,Albrecht_Dürer5.jpg,Albrecht Dürer,a drawing of a group of people in a room with ...,Life of the Virgin: The Death of the Virgin,en,Жизнь Девы: смерть Девы
5,Albrecht_Dürer6.jpg,Albrecht Dürer,a drawing of a man on a cross with a skull,Christ on the Cross,en,Христос на кресте
6,Albrecht_Dürer7.jpg,Albrecht Dürer,a drawing of a woman sitting at a table with a...,NaN,None,None
7,Albrecht_Dürer8.jpg,Albrecht Dürer,a drawing of a man and woman in a landscape wi...,The Promenade,en,Променад
8,Albrecht_Dürer9.jpg,Albrecht Dürer,a drawing of a woman holding a sword and a skull,The Coat-of-Arms with the Skull,en,Пальто с черепом
9,Albrecht_Dürer10.jpg,Albrecht Dürer,a drawing of a woman with a braided headband,Portrait of a young woman,en,Портрет молодой женщины


In [16]:
# Удаление колонки language, так как больше не нужна

df = df.drop(columns=['language'])

In [17]:
df = df.rename(columns={'Название картины': 'Название картины EN'})

In [18]:
df

,Название файла,Имя Художника,Описание картины,Название картины EN,Название картины RU
0,Albrecht_Dürer1.jpg,Albrecht Dürer,a drawing of a man with curly hair and a beard,Head of the Mark,Глава знака
1,Albrecht_Dürer2.jpg,Albrecht Dürer,a drawing of a woman holding a baby in her lap,Madonna and child on the grassy bank,Мадонна и ребёнок на банке травы
2,Albrecht_Dürer3.jpg,Albrecht Dürer,a drawing of a man and woman in a room with a ...,NaN,None
3,Albrecht_Dürer4.jpg,Albrecht Dürer,a drawing of a man standing next to a woman in...,Peter and John Healing the Cripple at the Gate...,Петр и Иоанн Исцеляли калеку у ворот храма
4,Albrecht_Dürer5.jpg,Albrecht Dürer,a drawing of a group of people in a room with ...,Life of the Virgin: The Death of the Virgin,Жизнь Девы: смерть Девы
...,...,...,...,...,...
11423,the-study-of-adam-1.jpg,Michelangelo,a drawing of a man sitting on a rock with his ...,The Study of Adam,Изучение Адама
11424,the-torment-of-saint-anthony.jpg,Michelangelo,painting of a man riding a dragon with a bird ...,The Torment of Saint Anthony,Наказание Святого Антония
11425,the-virgin-and-child-with-saint-john-and-angel...,Michelangelo,arafed painting of a woman reading to children...,The Virgin and Child with Saint John and Angel...,Дева и ребенок с святым Иоанном и ангелами (Ма...
11426,various-studies.jpg,Michelangelo,a drawing of a man holding a baby and a woman,Various studies,Различные исследования


In [19]:
# Сохранение датафрейма с новой колонкой

df.to_csv('/content/drive/MyDrive/full_df_EN_RU.csv', index=False)